In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as p
import collections
import uuid
import itertools
import json
import csv
from frgpascal.hardware.sampletray import SampleTray, AVAILABLE_VERSIONS as sampletray_versions
from frgpascal.hardware.liquidlabware import LiquidLabware, AVAILABLE_VERSIONS as liquid_labware_versions
from frgpascal.hardware.hotplate import AVAILABLE_VERSIONS as hotplate_versions
from frgpascal.experimentaldesign.helpers import where_to_store, calculate_mix, build_sample_list, generate_unique_id
from frgpascal.experimentaldesign.recipes import SolutionRecipe, SpincoatRecipe, AnnealRecipe, Sample
from frgpascal.experimentaldesign.tasks import *

# Placeholder Workflow

In [2]:
def build_solution_library(sample_list, stock_solutions):
    """
    finds volume required for each unique solution
    assigns a (labware, well) per solution
    generates a mixing netlist for liquid handler to convert stocks to mixed solutions
    """
    return solution_map, lh_netlist

In [3]:
def finalize_sample_list(sample_list, solution_map):
    """
    assigns the storage slots (tray, slot) and solution sources (labware, well)
    to each sample.
    """
    return final_sample_list, tray_list

# Define Labware

## Storage Trays

In [4]:
print('Available Versions:')
for v in sampletray_versions:
    print(f'\t{v}')

Available Versions:
	storage_v1
	storage_v2


In [5]:
sample_trays = [
    SampleTray(
        name='Tray1',
        version='storage_v1',
        gantry=None,
        gripper=None,
        p0=[0,0,0]
    ),
    SampleTray(
        name='Tray2',
        version='storage_v1',
        gantry=None,
        gripper=None,
        p0=[0,0,0]
    )
]

## Liquid Labware 
Versions are defined by the same json files used to define custom labware for the Opentrons2 liquid handler. 

New labware can be defined by following https://support.opentrons.com/en/articles/3136504-creating-custom-labware-definitions

In [6]:
print('Available Versions:')
for v in liquid_labware_versions:
    print(f'\t{v}')

Available Versions:
	frg_12_wellplate_15000ul
	frg_24_wellplate_4000ul
	frg_spincoater_v1
	greiner_96_wellplate_360ul
	sartorius_safetyspace_tiprack_200ul


Include _all possible_ liquid storage. Later on we will narrow this down to what is required once we know what solutions and volumes we need

In [7]:
solution_storage = [
    # LiquidLabware(
    #     name='96_Plate1',
    #     version='greiner_96_wellplate_360ul'
    # ),
    # LiquidLabware(
    #     name='96_Plate2',
    #     version='greiner_96_wellplate_360ul'
    # ),
    LiquidLabware(
        name='4mL_Tray1',
        version='frg_24_wellplate_4000ul'
    ),

    LiquidLabware(
        name='4mL_Tray2',
        version='frg_24_wellplate_4000ul'
    ),
    LiquidLabware(
        name='15mL_Tray1',
        version='frg_12_wellplate_15000ul'   
    ),
    LiquidLabware(
        name='15m_Tray2',
        version='frg_12_wellplate_15000ul'
    ),
]

#sort by volume,name
solution_storage.sort(key=lambda labware: labware.name)
solution_storage.sort(key=lambda labware: labware.volume)
print('Priority Fill Order:')
for ss in solution_storage: print(ss)

Priority Fill Order:
<LiquidLabware> 4mL_Tray1, 4.0 mL volume, 24 wells
<LiquidLabware> 4mL_Tray2, 4.0 mL volume, 24 wells
<LiquidLabware> 15mL_Tray1, 15.0 mL volume, 12 wells
<LiquidLabware> 15m_Tray2, 15.0 mL volume, 12 wells


## Define Experimental Mesh

Each of the following cells defines one dimension of the experiment. After defining the individual dimensions, a full sample list will be constructed using a user-selected Design of Expeiment (DOE)

### List of unique substrates. 
Eventually these may link to specific sample id's, but for now just set the unique types of substrates used

In [8]:
input_substrates = [
    '1mm_FTO'
]

### List of target solutions to spin cast. 

Solutions are defined using `SolutionRecipe` class instances.

Chemical formula is defined with underscores between each component. If no coefficient is provided, assumes =1. Example:

``` 
SolutionRecipe(
        solutes='MA_Pb_I3',
        solvent='DMF9_DMSO1',
        molarity=1
    )
```

In [9]:
target_solutions = [
    SolutionRecipe(
        solutes='MA_Pb_I3',
        solvent='DMF9_DMSO1',
        molarity=1
    ),
    # SolutionRecipe(
    #     solutes='FA_Pb_I3',
    #     solvent='DMF9_DMSO1',
    #     molarity=1
    # ),
    # SolutionRecipe(
    #     solutes='FA0.25_MA0.75_Pb_I3',
    #     solvent='DMF9_DMSO1',
    #     molarity=1
    # ),
    # SolutionRecipe(
    #     solutes='FA0.75_MA0.25_Pb_I3',
    #     solvent='DMF9_DMSO1',
    #     molarity=1
    # ),
    # SolutionRecipe(
    #     solutes='MA0.5_FA0.5_Pb_I3',
    #     solvent='DMF9_DMSO1',
    #     molarity=1
    # ),
]

### Spincoating Recipes


#### Defining Antisolvents
Antisolvents are also defined using `SolutionRecipe` class instances. Solutes are left empty, and molarity is unused so can be left as 1/whatever number you like.

Example:
``` 
SolutionRecipe(
        solutes='',
        solvent='Chlorobenzene',
        molarity=1
    )
```

Note that these do not have to be stored in a list, as antisolvents are not considered a standalone experimental dimension. We will include these as spincoating parameters, which along with spin speeds etc. form an experimental dimension that can be permuted/whatever

In [10]:
cb_antisolvent = SolutionRecipe(
            solutes='',
            solvent='Chlorobenzene',
            molarity=1
        )
# ma_antisolvent = SolutionRecipe(
#             solutes='',
#             solvent='MethylAcetate',
#             molarity=1
#         )


#### Defining spincoating parameters

Spincoating recipes are defined using the `SpincoatRecipe` object. This consists of a nested list of [speed, acceleration, duration] spin steps. The `SolutionRecipe`, volume, and droptime of both the initial solution and optional antisolvent are also defined. Drop times are relative to t=0. Static spincoating (dispense before spin start) can be achieved by setting a negative number for drop time. Example:

```
SpincoatRecipe(
        steps=[
            [500,1000,20], #speed (rpm), acceleration (rpm/s), duration (s)
            [4000,4000,30]
        ],
        solution=SolutionRecipe(...), #this would usually be defined earlier
        solution_volume=100, #uL
        solution_droptime=-5, #seconds relative to start time
        antisolvent=SolutionRecipe(...), #also defined earlier
        antisolvent_volume=50, #seconds relative to start time
        antisolvent_droptime=40,
    )
```

the initial solution is typically left out here, as it will be filled in with the `target_solutions` defined earlier when building the DOE. Example:

```
SpincoatRecipe(
        steps=[
            [500,1000,20], #speed (rpm), acceleration (rpm/s), duration (s)
            [4000,4000,30]
        ],
        solution_volume=100, #uL
        solution_droptime=-5, #seconds relative to start time
        antisolvent=SolutionRecipe(...), #also defined earlier
        antisolvent_volume=50, #seconds relative to start time
        antisolvent_droptime=40,
    )
```

In the case that no antisolvent is needed, the three antisolvent terms can be left out.

```
SpincoatRecipe(
        steps=[
            [500,1000,20], #speed (rpm), acceleration (rpm/s), duration (s)
            [4000,4000,30]
        ],
        solution=SolutionRecipe(...), #this would usually be defined earlier
        solution_volume=100, #uL
        solution_droptime=-5, #seconds relative to start time
    )
```

In [11]:
spincoat_recipes = [
    SpincoatRecipe(
        steps=[
            [500,1000,20], #speed (rpm), acceleration (rpm/s), duration (s)
            [4000,4000,30]
        ],
        solution_volume=20, #uL
        solution_droptime=-5, #seconds relative to start time
        antisolvent=cb_antisolvent,
        antisolvent_volume=40, #seconds relative to start time
        antisolvent_droptime=40,
    ),
    SpincoatRecipe(
        steps=[
            [500,1000,20],
            [4000,4000,30]
        ],
        solution_volume=20, #uL
        solution_droptime=-5,
        antisolvent=cb_antisolvent,
        antisolvent_volume=40,
        antisolvent_droptime=45,
    ),
]


#### Annealing Conditions
Annealing conditions are defined as a list of `AnnealRecipe` class instances. Only two parameters - temperature (C) and duration (s)

Example:
``` 
    AnnealRecipe(
        temperature=100, #degrees C
        duration=60*60, #seconds
    ),
```

In [12]:
anneal_recipes = [
    AnnealRecipe(
        temperature=100, #degrees C
        duration=4*60, #seconds
    ),
    # AnnealRecipe(
    #     temperature=120, #degrees C
    #     duration=60*40, #seconds
    # ),
    # AnnealRecipe(
    #     temperature=140, #degrees C
    #     duration=60*20, #seconds
    # ),
]

## Design of Experiment

currently only a full-factorial design is implemented. arguments are lists defined above, and n_repeats is the number of times to repeat each unique combination of conditions

In [13]:
samples = build_sample_list(
    available_trays = sample_trays,
    input_substrates = input_substrates,
    target_solutions = target_solutions,
    spincoat_recipes=spincoat_recipes,
    anneal_recipes=anneal_recipes,
    n_repeats=2
)
unique_solutions = list(set(
    [s.spincoat_recipe.solution for s in samples]
    + [s.spincoat_recipe.antisolvent for s in samples]))

print(f'{len(unique_solutions)} unique solutions required across {len(samples)} planned samples')

2 unique solutions required across 4 planned samples


# Preparing the Solutions

## Define Stock Solutions

These are the solutions that you will be mixing to (hopefully) cover your target solutions. If they don't work, the code will tell you, and you can come back to this point and try again.

In [14]:
stock_solutions = [
    SolutionRecipe(
        solutes='MA_Pb_I3',
        solvent='DMF9_DMSO1',
        molarity=1
    ),
    SolutionRecipe(
        solutes='FA_Pb_I3',
        solvent='DMF9_DMSO1',
        molarity=1
    ),
    SolutionRecipe(
        solutes='',
        solvent='DMF',
        molarity=1
    ),
    SolutionRecipe(
        solutes='',
        solvent='DMSO',
        molarity=1
    ),
    SolutionRecipe(
        solutes='',
        solvent='Chlorobenzene',
        molarity=1
    ),
    SolutionRecipe(
        solutes='',
        solvent='MethylAcetate',
        molarity=1
    ),
]

Can we cover target solutions with these stocks? and if so, how much of each stock to mix for each target?

In [15]:
min_volume = 0
solution_details = {s:dict(volume=min_volume, labware=None, slot=None) for s in unique_solutions}
for s in samples:
    for sol in [s.spincoat_recipe.solution, s.spincoat_recipe.antisolvent]:
        if sol is not None:
            solution_details[sol]['volume'] += s.spincoat_recipe.solution_volume
    

In [16]:
mixtures = np.array([
    calculate_mix(
        target=solution,
        volume=v['volume'],
        stock_solutions = stock_solutions
    ) for solution, v in solution_details.items()])

In [17]:
volumes_needed_per_stock = {
    soln: vol for soln,vol in zip(stock_solutions,mixtures.sum(axis=0))}
for s, v in volumes_needed_per_stock.items():
    if v > 0:
        print(f'{v/1e3} mL of {s}')

0.08 mL of <SolutionRecipe> 1M MA_Pb_I3 in DMF9_DMSO1
0.08 mL of <SolutionRecipe> Chlorobenzene


Which solutions need to be made from stocks?

In [18]:
to_be_mixed = {s:v for s,v in solution_details.items() if s not in stock_solutions}

In [19]:
for solution, v in solution_details.items():
    ll = where_to_store(v['volume'], solution_storage) #which liquid labware
    slot = ll.load(solution)
    solution_details[solution]['labware'] = ll.name
    solution_details[solution]['slot'] = slot

In [20]:
for s in samples:
    solution = s.spincoat_recipe.solution
    d = solution_details[solution]
    solution.well = {
        'tray': d['labware'],
        'slot': d['slot'],
    }

    antisolvent = s.spincoat_recipe.antisolvent
    if antisolvent is not None:
        d = solution_details[antisolvent]
        antisolvent.well = {
            'tray': d['labware'],
            'slot': d['slot'],
        }

Set up the mixing netlist for liquid handler

In [21]:
mixing_netlist = {} #destination: {source, volume}

for target, target_v in solution_details.items():
    if target in stock_solutions:
        continue #stock solution is already made
    destination = f"{target_v['labware']}-{target_v['slot']}"
    mixing_netlist[destination] = {}
    required_volumes = calculate_mix(
        target=target,
        volume=target_v['volume'],
        stock_solutions = stock_solutions
    )
    for stock, vol in zip(stock_solutions, required_volumes):
        if vol > 0:    
            stock_v = solution_details[stock]
            source = f"{stock_v['labware']}-{stock_v['slot']}"
            mixing_netlist[destination][source]=vol
            


## Build the Experiment Schedule

In [22]:
for s in samples:
    s.tasks = generate_tasks_for_sample(s)

In [23]:
scheduler = Scheduler(
    samples=samples,
    spanning_tasks = [
        (StorageToSpincoater, SpincoaterToHotplate),
        (StorageToCharacterization, CharacterizationToStorage),
    ] #these tasks spanning each pair must be done in sequence per sample
)
scheduler.solve(5) #argument defines how long (s) to work on finding a solution

In [24]:
ordered_tasks = [
    task
    for sample in samples
    for task in sample.tasks
]
ordered_tasks.sort(key=lambda x:x.start)

In [25]:
for precedent, task in zip(ordered_tasks, ordered_tasks[1:]):
    if precedent.start == task.start:
        task.precedents = precedent.precedents
    elif (precedent.task in ['anneal', 'cooldown']) and (precedent.sample != task.sample):
        continue
    elif precedent not in [p[0] for p in task.precedents]:
        task.precedents.append((precedent, False))


# Write final files to disk

## Write mixing netlist to disk, will be copy-pasted into listener.py protocol

In [26]:
with open('ot2netlist.json', 'w') as f:
    json.dump(mixing_netlist, f, indent=4, sort_keys=True)
print('Liquid Handler Netlist String dumped to "ot2netlist.json"')

Liquid Handler Netlist String dumped to "ot2netlist.json"


## Maestro netlist to disk, will be input to maestro to execute experiment

In [27]:
ordered_task_output = [
    task.to_dict() for task in ordered_tasks
]
samples_output = {sample.name:sample.to_dict() for sample in samples}
out = {
    'samples': samples_output,
    'tasks': ordered_task_output,
}
with open('maestronetlist.json', 'w') as f:
    json.dump(out, f, indent=4, sort_keys=True)
print('Maestro Netlist dumped to "maestronetlist.json"')

Maestro Netlist dumped to "maestronetlist.json"


## Solution Instructions to csv, tells operator where/how much to put solutions

In [28]:
with open('stock_solutions.csv', 'w', newline='') as f:
    writer = csv.writer(f, delimiter=',')
    header = [
        'Tray',
        'Slot',
        'Volume (uL)',
        'Solutes',
        'Molarity (M)',
        'Solvent',        
    ]
    writer.writerow(header)
    for solution, details in solution_details.items():
        volume = volumes_needed_per_stock.get(solution, 'Empty Vial')    
        line = [
            details['labware'],
            details['slot'],
            volume,
            solution.solutes,
            solution.molarity,
            solution.solvent,
        ]
        writer.writerow(line)